In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold

In [2]:
with xr.open_dataset("../cpcb_17_to_23.nc") as ds:
    pass
ds

<xarray.Dataset>
Dimensions:      (time: 61344, station: 65)
Coordinates:
  * time         (time) datetime64[ns] 2017-01-01T00:30:00 ... 2023-12-31T23:...
  * station      (station) object 'Alipur_Delhi_DPCC' ... 'Wazirpur_Delhi_DPCC'
    lat          (station) float64 ...
    lon          (station) float64 ...
    elevation    (station) float64 ...
    pop_1km      (station) float64 ...
    pop_2km      (station) float64 ...
    pop_3km      (station) float64 ...
Data variables: (12/24)
    PM2.5        (time, station) float64 ...
    PM10         (time, station) float64 ...
    NO           (time, station) float64 ...
    NO2          (time, station) float64 ...
    NOx          (time, station) float64 ...
    NH3          (time, station) float64 ...
    ...           ...
    WD           (time, station) float64 ...
    RF           (time, station) float64 ...
    TOT-RF       (time, station) float64 ...
    SR           (time, station) float64 ...
    BP           (time, station) float64 ...
    VWS          (time, station) float64 ...

In [3]:
ds['PM2.5'].resample(time='Y').mean().isnull().sum(dim='station').to_dataframe()

,PM2.5
time,
2017-12-31,42
2018-12-31,18
2019-12-31,8
2020-12-31,2
2021-12-31,1
2022-12-31,0
2023-12-31,1


From 2020 data seems good. Now, we check monthly.

In [4]:
ds['PM2.5'].sel(time=slice('2020','2023')).resample(time='M').mean().isnull().sum(dim='station').to_dataframe()

,PM2.5
time,
2020-01-31,9
2020-02-29,10
2020-03-31,5
2020-04-30,5
2020-05-31,6
2020-06-30,5
2020-07-31,4
2020-08-31,4
2020-09-30,4


In [5]:
print("2020")
for month in range(7, 13):
    tmp_ds = ds['PM2.5'].sel(time='2020-'+str(month).zfill(2)).resample(time='M').mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())

print("\n2021")
for month in range(1, 13):
    tmp_ds = ds['PM2.5'].sel(time='2021-'+str(month).zfill(2)).resample(time='M').mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())
    
print("\n2022")
for month in range(1, 13):
    tmp_ds = ds['PM2.5'].sel(time='2022-'+str(month).zfill(2)).resample(time='M').mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())
    
print("\n2023")
for month in range(1, 13):
    tmp_ds = ds['PM2.5'].sel(time='2023-'+str(month).zfill(2)).resample(time='M').mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())

2020
7 ['Burari_Crossing_Delhi_IMD', 'Chandni_Chowk_Delhi_IITM', 'Lodhi_Road_Delhi_IITM', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
8 ['Burari_Crossing_Delhi_IMD', 'Chandni_Chowk_Delhi_IITM', 'Lodhi_Road_Delhi_IITM', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
9 ['Burari_Crossing_Delhi_IMD', 'Chandni_Chowk_Delhi_IITM', 'Lodhi_Road_Delhi_IITM', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
10 ['Burari_Crossing_Delhi_IMD', 'Chandni_Chowk_Delhi_IITM', 'Lodhi_Road_Delhi_IITM', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
11 ['Burari_Crossing_Delhi_IMD', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
12 ['Burari_Crossing_Delhi_IMD', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']

2021
1 ['Burari_Crossing_Delhi_IMD', 'Narela_Delhi_DPCC', 'Sardar_Patel_Inter_College_Baghpat_UPPCB', 'Sector_11_Faridabad_HSPCB']
2 ['Burari_Crossing_Delhi_IMD', 'Narela_Delhi_DPCC', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
3 ['Burari_Crossing_Delhi_IMD', 'Sardar_Patel_Inter_College_Baghpat_UPPCB']
4 ['Burari_Crossing

In [6]:
missing_df = ds[['PM2.5', 'CO']].sel(time=slice('2022', '2023')).isnull().mean(dim='time').to_dataframe().sort_values('PM2.5', ascending=False).head(16)
missing_df

,PM2.5,CO,lat,lon,elevation,pop_1km,pop_2km,pop_3km
station,,,,,,,,
New_Collectorate_Baghpat_UPPCB,0.957477,0.960959,28.974801,77.213357,235.851746,3485.0,11088.0,19118.0
Lodhi_Road_Delhi_IITM,0.688984,0.843664,28.588333,77.221667,232.633987,69984.0,202022.0,512062.0
Chandni_Chowk_Delhi_IITM,0.606164,0.619863,28.656756,77.227234,242.798279,99175.0,555126.0,969081.0
Murthal_Sonipat_HSPCB,0.316495,0.375970,29.027200,77.062100,236.980927,3109.0,15336.0,28861.0
Sector_16A_Faridabad_HSPCB,0.277740,0.520947,28.408842,77.309908,225.533722,56423.0,243838.0,463028.0
Sardar_Patel_Inter_College_Baghpat_UPPCB,0.240753,0.314612,28.964949,77.278761,245.757690,2999.0,32177.0,55897.0
Vikas_Sadan_Gurugram_HSPCB,0.236016,0.224087,28.450124,77.026305,245.607208,54325.0,203621.0,374025.0
Burari_Crossing_Delhi_IMD,0.222888,0.104566,28.725650,77.201157,225.292328,9881.0,122118.0,314713.0
Sector_30_Faridabad_HSPCB,0.204795,0.188242,28.441700,77.321700,224.178101,18856.0,105891.0,224637.0


In [7]:
clean_ds = ds.sel(station=~ds.station.isin(missing_df.index))
# print(clean_ds)
freq = "M"
# print("2020")
# for month in range(7, 13):
#     tmp_ds = clean_ds['PM2.5'].sel(time='2020-'+str(month).zfill(2)).resample(time=freq).mean().isnull().squeeze()
#     print(month, tmp_ds[tmp_ds].station.values.tolist())

# print("\n2021")
# for month in range(1, 13):
#     tmp_ds = clean_ds['PM2.5'].sel(time='2021-'+str(month).zfill(2)).resample(time=freq).mean().isnull().squeeze()
#     print(month, tmp_ds[tmp_ds].station.values.tolist())
    
print("\n2022")
for month in range(1, 13):
    tmp_ds = clean_ds['PM2.5'].sel(time='2022-'+str(month).zfill(2)).resample(time=freq).mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())
    
print("\n2023")
for month in range(1, 13):
    tmp_ds = clean_ds['PM2.5'].sel(time='2023-'+str(month).zfill(2)).resample(time=freq).mean().isnull().squeeze()
    print(month, tmp_ds[tmp_ds].station.values.tolist())


2022
1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
12 []

2023
1 []
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
12 []


In [8]:
final_ds = clean_ds.sel(time=slice('2022', '2023'))
missing = final_ds['PM2.5'].isnull()
print(missing.sum())
# fill data with linear interpolation over time
final_ds['PM2.5'] = final_ds['PM2.5'].interpolate_na(dim='time', method='linear', fill_value="extrapolate")
print(final_ds['PM2.5'].isnull().sum())
final_ds["PM2.5_missing"] = missing
final_ds

<xarray.DataArray 'PM2.5' ()>
array(39894)
<xarray.DataArray 'PM2.5' ()>
array(0)


<xarray.Dataset>
Dimensions:        (time: 17520, station: 49)
Coordinates:
  * time           (time) datetime64[ns] 2022-01-01T00:30:00 ... 2023-12-31T2...
  * station        (station) object 'Alipur_Delhi_DPCC' ... 'Wazirpur_Delhi_D...
    lat            (station) float64 28.82 28.65 28.67 28.7 ... 28.66 28.67 28.7
    lon            (station) float64 77.15 77.32 76.93 ... 77.36 77.32 77.17
    elevation      (station) float64 232.7 228.1 235.9 ... 224.5 227.6 236.9
    pop_1km        (station) float64 5.191e+03 1.125e+05 ... 1.575e+05 1.543e+05
    pop_2km        (station) float64 2.459e+04 3.452e+05 ... 4.835e+05 5.677e+05
    pop_3km        (station) float64 5.667e+04 9.286e+05 ... 1.226e+06 1.084e+06
Data variables: (12/25)
    PM2.5          (time, station) float64 219.5 613.2 135.1 ... 207.8 213.8
    PM10           (time, station) float64 ...
    NO             (time, station) float64 ...
    NO2            (time, station) float64 ...
    NOx            (time, station) float64 ...
    NH3            (time, station) float64 ...
    ...             ...
    RF             (time, station) float64 ...
    TOT-RF         (time, station) float64 ...
    SR             (time, station) float64 ...
    BP             (time, station) float64 ...
    VWS            (time, station) float64 ...
    PM2.5_missing  (time, station) bool False False False ... False False False

In [9]:
final_ds.to_netcdf("data.nc")

In [10]:
stations = final_ds.station.values

for fold_id, (train_stations_idx, test_stations_idx) in enumerate(KFold(n_splits=3, random_state=2024, shuffle=True).split(stations)):
    train_stations = stations[train_stations_idx]
    test_stations = stations[test_stations_idx]
    np.save(f"fold_{fold_id}_train.npy", train_stations)
    np.save(f"fold_{fold_id}_test.npy", test_stations)

/home/patel_zeel/miniconda3/envs/torch_gpu_py311/lib/python3.11/site-packages/numpy/lib/format.py:362: UserWarning: metadata on a dtype is not saved to an npy/npz. Use another format (such as pickle) to store it.
  d['descr'] = dtype_to_descr(array.dtype)
